# Quick exploration of trends

In [1]:
# import libraries for working with .nc
import xarray as xr
import os   
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# load the 3days binned data "/mnt/d/Dropbox/R_projects/SV_phenology/test_chl3days.nc")
ds = xr.open_dataset("/mnt/d/Dropbox/R_projects/SV_phenology/test_chl3days.nc")


In [3]:
ds

<xarray.Dataset>
Dimensions:    (time: 62, latitude: 1248, longitude: 1056, time2: 62)
Coordinates:
  * latitude   (latitude) float64 84.99 84.98 84.97 84.96 ... 72.03 72.02 72.01
  * longitude  (longitude) float64 -4.995 -4.943 -4.891 ... 49.85 49.9 49.95
  * time       (time) datetime64[ns] 2010-03-21 2010-03-24 ... 2010-09-20
  * time2      (time2) timedelta64[ns] 14689 days 14692 days ... 14872 days
Data variables:
    CHL        (time, latitude, longitude) float32 ...
    mask       (latitude, longitude) float64 ...

In [6]:
coarse_ds = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').mean()
coarse_ds

<xarray.DataArray 'CHL' (time: 62, latitude: 8, longitude: 8)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [0.12729791, 0.09606392, 0.08946232, ...,        nan,
                nan, 0.24735555],
        [0.13274297, 0.11526578, 0.09735166, ...,        nan,
                nan, 0.28803653],
        [0.11525264, 0.10396857, 0.0999292 , ..., 0.13823266,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [       nan,        nan, 0.8068721 , ..., 1.9054614 ,
         0.61815464, 0.71077305],
        [       nan,        nan,        nan, ...,        nan,
         1.2730025 , 0.74816614],
        [       nan,        nan, 0.7938476 , ..., 0.5548673 ,
         0.45855075, 0.36725056]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.20832567],
        [       nan,        nan,        nan, ...,        nan,
                nan, 1.5242916 ],
        [       nan, 0.29074362, 0.67132705, ..., 1.1387384 ,
                nan,        nan]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 84.19 82.56 80.94 79.31 ... 76.06 74.44 72.81
  * longitude  (longitude) float64 -1.583 5.292 12.17 ... 32.79 39.67 46.54
  * time       (time) datetime64[ns] 2010-03-21 2010-03-24 ... 2010-09-20
Attributes:
    long_name:          Chlorophyll-a concentration in sea water using the OC...
    units:              milligram m-3
    standard_name:      mass_concentration_of_chlorophyll_a_in_sea_water
    units_nonstandard:  mg m^-3
    type:               surface
    source:             ENVISAT,Aqua,Orbview-2/SeaStar-MERIS,MODISA,SeaWiFS-L3
    valid_max:          66.83439
    valid_min:          0.01

In [10]:
# transform the data to table
table = coarse_ds.to_dataframe()
table

CHL
time       latitude longitude           
2010-03-21 84.1875  -1.583333        NaN
                     5.291667        NaN
                     12.166667       NaN
                     19.041667       NaN
                     25.916667       NaN
...                                  ...
2010-09-20 72.8125   19.041667  0.835593
                     25.916667  1.072999
                     32.791667  1.138738
                     39.666667       NaN
                     46.541667       NaN

[3968 rows x 1 columns]

In [11]:
# same but I want sd
ds_coarseSD = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').std()

/home/matzuc/miniconda3/envs/geo/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [15]:
# create data frame and attach to table
tableSD = ds_coarseSD.to_dataframe()
# rename the column to CHL_std
tableSD = tableSD.rename(columns={'CHL': 'CHL_std'})



tab2 = pd.concat([table, tableSD], axis=1)
tab2

CHL   CHL_std
time       latitude longitude                     
2010-03-21 84.1875  -1.583333        NaN       NaN
                     5.291667        NaN       NaN
                     12.166667       NaN       NaN
                     19.041667       NaN       NaN
                     25.916667       NaN       NaN
...                                  ...       ...
2010-09-20 72.8125   19.041667  0.835593  0.410189
                     25.916667  1.072999  0.341559
                     32.791667  1.138738  0.314777
                     39.666667       NaN       NaN
                     46.541667       NaN       NaN

[3968 rows x 2 columns]

In [17]:
# same but with n of not nan values
ds_coarseNAN = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').count()

tabNAN = ds_coarseNAN.to_dataframe()
tabNAN = tabNAN.rename(columns={'CHL': 'CHL_n'})

tab2 = pd.concat([tab2, tabNAN], axis=1)
tab2



CHL   CHL_std  CHL_n  CHL_n
time       latitude longitude                                   
2010-03-21 84.1875  -1.583333        NaN       NaN      0      0
                     5.291667        NaN       NaN      0      0
                     12.166667       NaN       NaN      0      0
                     19.041667       NaN       NaN      0      0
                     25.916667       NaN       NaN      0      0
...                                  ...       ...    ...    ...
2010-09-20 72.8125   19.041667  0.835593  0.410189   3590   3590
                     25.916667  1.072999  0.341559   4584   4584
                     32.791667  1.138738  0.314777     10     10
                     39.666667       NaN       NaN      0      0
                     46.541667       NaN       NaN      0      0

[3968 rows x 4 columns]

In [20]:
# write csv to data/test_chl3days.csv
tab2.to_csv("/mnt/d/Dropbox/R_projects/SV_phenology/data/test_chl3days.csv")

In [23]:
# tab2 2 write time
tab2 = tab2.reset_index()
tab2 = tab2.rename(columns={'time': 'date'})
tab2

,date,latitude,longitude,CHL,CHL_std,CHL_n,CHL_n
0,2010-03-21,84.1875,-1.583333,NaN,NaN,0,0
1,2010-03-21,84.1875,5.291667,NaN,NaN,0,0
2,2010-03-21,84.1875,12.166667,NaN,NaN,0,0
3,2010-03-21,84.1875,19.041667,NaN,NaN,0,0
4,2010-03-21,84.1875,25.916667,NaN,NaN,0,0
...,...,...,...,...,...,...,...
3963,2010-09-20,72.8125,19.041667,0.835593,0.410189,3590,3590
3964,2010-09-20,72.8125,25.916667,1.072999,0.341559,4584,4584
3965,2010-09-20,72.8125,32.791667,1.138738,0.314777,10,10
3966,2010-09-20,72.8125,39.666667,NaN,NaN,0,0


In [28]:
# remove all observations from tab2 maintaning only the header
tab3 = tab2.iloc[0:0]
tab3

,date,latitude,longitude,CHL,CHL_std,CHL_n,CHL_n


siccome sembra funzionare, faccio i conti per tutto il periodo

In [41]:

outp = '/mnt/d/Dropbox/R_projects/SV_phenology/data/daily_coarsened/'

# loop for each year within 1998 and 2022
for year in range(1998,2023):
    # loop for each month within march and october

    # list all files
    tpath = os.path.join(outp + '{}'.format(year))
    files = [os.path.join(tpath , f) for f in os.listdir(tpath)]

    # use xarray to stack all files in files
    ds = xr.open_mfdataset(files, combine='by_coords')
    
    
    # mean
    dsCHLA = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').mean()
    ttab = dsCHLA.to_dataframe()
    dsCHLA.close()
    
    #sd
    sdSD = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').std()
    sdTAB = sdSD.to_dataframe()
    sdTAB = sdTAB.rename(columns={'CHL': 'CHL_std'})
    ttab = pd.concat([ttab, sdTAB], axis=1)
    
    # n
    dsN = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').count()
    nTAB = dsN.to_dataframe()
    nTAB = nTAB.rename(columns={'CHL': 'CHL_n'})
    ttab = pd.concat([ttab, nTAB], axis=1)
    
    ttab = ttab.reset_index()
    ttab = ttab.rename(columns={'time': 'date'})
    

    # write like tab2.to_csv("/mnt/d/Dropbox/R_projects/SV_phenology/data/test_chl3days.csv") but with the name tab_8x8_year.csv
    ttab.to_csv("/mnt/d/Dropbox/R_projects/SV_phenology/data/tab_8x8_{}.csv".format(year))

                     



    # close the dataset
    ds.close()


    # print the year and month
    print(year)
        

/tmp/ipykernel_618/1394606149.py:16: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  dsCHLA = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').mean()
/tmp/ipykernel_618/1394606149.py:21: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
   

1998


/tmp/ipykernel_618/1394606149.py:16: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  dsCHLA = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').mean()
/tmp/ipykernel_618/1394606149.py:21: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
   

1999


/tmp/ipykernel_618/1394606149.py:16: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  dsCHLA = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').mean()
/tmp/ipykernel_618/1394606149.py:21: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
   

...

2020


/tmp/ipykernel_618/1394606149.py:16: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  dsCHLA = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').mean()
/tmp/ipykernel_618/1394606149.py:21: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
   

2021


/tmp/ipykernel_618/1394606149.py:16: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  dsCHLA = ds['CHL'].coarsen(latitude=ds.dims['latitude']//8, longitude=ds.dims['longitude']//8, boundary='trim').mean()
/tmp/ipykernel_618/1394606149.py:21: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
   

2022


In [38]:
ttab

,date,latitude,longitude,CHL,CHL_std,CHL_n
0,1998-03-01,84.1875,-1.583333,NaN,NaN,0
1,1998-03-01,84.1875,5.291667,NaN,NaN,0
2,1998-03-01,84.1875,12.166667,NaN,NaN,0
3,1998-03-01,84.1875,19.041667,NaN,NaN,0
4,1998-03-01,84.1875,25.916667,NaN,NaN,0
...,...,...,...,...,...,...
15611,1998-10-31,72.8125,19.041667,NaN,NaN,0
15612,1998-10-31,72.8125,25.916667,NaN,NaN,0
15613,1998-10-31,72.8125,32.791667,NaN,NaN,0
15614,1998-10-31,72.8125,39.666667,NaN,NaN,0


In [39]:
tab3

,date,latitude,longitude,CHL,CHL_std,CHL_n,CHL_n
